In [1]:
import pysam
import re
from collections import defaultdict
import pandas as pd
import altair as alt

In [19]:
# --- Step 1: Parse haplotype frequencies from FASTA ---
hap_freqs = {}

In [20]:

with open("haps.final.fasta", "r") as fasta:
    for line in fasta:
        if line.startswith(">"):
            # Example header: >path24 3770x frequency=0.027
            match = re.match(r'^>(\S+).*frequency=([\d.]+)', line)
            if match:
                hap_id = match.group(1)
                freq = float(match.group(2))
                hap_freqs[hap_id] = freq

In [21]:
hap_freqs

{'path1': 0.087,
 'path2': 0.041,
 'path4': 0.063,
 'path6': 0.065,
 'path8': 0.024,
 'path9': 0.077,
 'path10': 0.017,
 'path11': 0.06,
 'path12': 0.042,
 'path13': 0.017,
 'path14': 0.032,
 'path15': 0.051,
 'path16': 0.09,
 'path17': 0.043,
 'path18': 0.016,
 'path19': 0.043,
 'path20': 0.053,
 'path22': 0.058,
 'path23': 0.05,
 'path24': 0.027,
 'path26': 0.044}

In [22]:
# --- Step 2: Process SAM with pysam ---
samfile = pysam.AlignmentFile("output.sam", "r")

hap_to_lineage = {}

for read in samfile.fetch(until_eof=True):
    if read.is_unmapped or read.is_supplementary or read.is_secondary:
        continue
    hap_id = read.query_name
    lineage = samfile.get_reference_name(read.reference_id)
    hap_to_lineage[hap_id] = lineage

samfile.close()


In [23]:
hap_to_lineage

{'path1': 'HXB2',
 'path6': 'YU2',
 'path11': '896',
 'path15': '896',
 'path22': 'HXB2',
 'path23': 'YU2',
 'path26': 'JRCSF'}

In [24]:
# --- Step 3: Sum frequencies per lineage ---
lineage_abundance = defaultdict(float)

for hap_id, lineage in hap_to_lineage.items():
    freq = hap_freqs.get(hap_id)
    if freq is not None:
        lineage_abundance[lineage] += freq


In [25]:
# --- Output ---
print("Lineage Abundances:")
for lineage, abundance in sorted(lineage_abundance.items()):
    print(f"{lineage}: {abundance:.4f}")

Lineage Abundances:
896: 0.1110
HXB2: 0.1450
JRCSF: 0.0440
YU2: 0.1150


In [26]:

#fractions in the truth
truth = {"896": 0.226, "HXB2": 0.10, "JRCSF": 0.296, "NL43": 0.269, "YU2": 0.109}

In [27]:
truth


{'896': 0.226, 'HXB2': 0.1, 'JRCSF': 0.296, 'NL43': 0.269, 'YU2': 0.109}

In [28]:
lineage_abundance

defaultdict(float,
            {'HXB2': 0.145,
             'YU2': 0.115,
             '896': 0.11099999999999999,
             'JRCSF': 0.044})

In [29]:
lineage_abundance["NL43"] = 0.0

In [30]:
scatterplot_data = pd.DataFrame({
    'Actual': list(truth.values()),
    'Predicted': list(lineage_abundance.values()),
    })

In [31]:
def create_scatter_plot(data):
    scatterplot = alt.Chart(data).mark_point().transform_calculate(jitter="random()").encode(
        x='Actual:Q',
        y='Predicted:Q',
        color=alt.value("black"),
        tooltip=alt.Tooltip(['Actual', 'Predicted'])
    )

    # add a diagonal line
    line = pd.DataFrame({
    'Actual': [0.0, 0.4],
    'Predicted':  [0.0, 0.4],
    })

    line_plot = alt.Chart(line).mark_line(color='red').encode(
    x= alt.X('Actual',scale=alt.Scale(domain=[0.0, 0.4])),
    y= alt.Y('Predicted', scale=alt.Scale(domain=[0.0, 0.4])),
    )

    plot = (scatterplot + line_plot).properties(
        width=200,
        height=200
    )
    return plot

In [32]:
plot = create_scatter_plot(scatterplot_data)

In [33]:
plot

alt.LayerChart(...)

In [34]:
#export to svg and html
plot.save("scatterplot.svg")